In [ ]:
# hide

%load_ext nb_black

<IPython.core.display.Javascript object>

In [ ]:
# default_exp fastapi.main

<IPython.core.display.Javascript object>

## fastAPI Application Views

```
uvicorn will_it_saturate.fastapi.main:app --reload
```

In [ ]:
# export
import time

from pathlib import Path

from fastapi import FastAPI
from fastapi.responses import FileResponse

<IPython.core.display.Javascript object>

In [ ]:
# export

app = FastAPI()


import aiofiles

from starlette.types import Receive, Scope, Send


class MyFileResponse(FileResponse):
    async def __call__(self, scope: Scope, receive: Receive, send: Send) -> None:
        if self.stat_result is None:
            try:
                stat_result = await aio_stat(self.path)
                self.set_stat_headers(stat_result)
            except FileNotFoundError:
                raise RuntimeError(f"File at path {self.path} does not exist.")
            else:
                mode = stat_result.st_mode
                if not stat.S_ISREG(mode):
                    raise RuntimeError(f"File at path {self.path} is not a file.")
        await send(
            {
                "type": "http.response.start",
                "status": self.status_code,
                "headers": self.raw_headers,
            }
        )
        if self.send_header_only:
            await send({"type": "http.response.body", "body": b"", "more_body": False})
        else:
            async with aiofiles.open(self.path, mode="rb") as file:
                start = time.perf_counter()
                more_body = True
                while more_body:
                    chunk = await file.read(self.chunk_size)
                    more_body = len(chunk) == self.chunk_size
                    await send(
                        {
                            "type": "http.response.body",
                            "body": chunk,
                            "more_body": more_body,
                        }
                    )
                elapsed = s - time.perf_counter()
                print(f"elapsed: {elapsed} for {self.path}")
                    
        if self.background is not None:
            await self.background()



@app.get("/")
def read_root():
    return {"Hello": "World"}


@app.get("/pathinfo/{base_dir}")
def get_file_urls(base_dir: str):
    pass


@app.get("/{base}/{path}/{num}")
async def serve_file(base: str, path: str, num: int):
    file_path = str(Path(base) / path / str(num))
    return MyFileResponse(file_path)

In [ ]:
# hide
# dont_test

from nbdev.export import notebook2script

notebook2script()

Converted 00_core.ipynb.
Converted 01_django_views.ipynb.
Converted 01_fastapi_views.ipynb.
Converted 03_nginx_docker_server.ipynb.
Converted 03_servers.ipynb.
Converted 04_benchmark_clients.ipynb.
Converted 05_persistence.ipynb.
Converted 06_run_benchmark.ipynb.
Converted django_start_server.ipynb.
Converted index.ipynb.
Converted run_wrk_benchmark_test.ipynb.
Converted use_nginx_to_serve_files_test.ipynb.
